# Titanic Survival Classification - Genetic Programming

So for this notebook I will be developing a simple genetic algorithm from scratch in order to understand the underlying principles of how it works for use later on.

The inspiration for this approach is to make something approaching - 

https://www.kaggle.com/scirpus/genetic-programming-lb-0-88

While there are multiple libraries that achieve everything in this notebook and more in a more efficient way, the purpose of this is to get a grasp on how genetic programming works, so that when I come to use libraries like DEAP in the future I will have a greater understanding.

Sources - 

https://blog.sicara.com/getting-started-genetic-algorithms-python-tutorial-81ffa1dd72f9

First up do the usual imports.

In [1]:
##### First importing some relevant packages
import numpy as np
import pandas as pd

#Stop pandas from truncating output view
pd.options.display.max_columns = None

#Import Tensorflow
import tensorflow as tf

#Import Keras
from keras import layers
from keras.layers import Input, Dense, Activation, BatchNormalization, Dropout, Reshape, Flatten
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.models import Sequential, Model
from keras import regularizers
from keras.optimizers import Adam, SGD

#Import mathematical functions
import math
import matplotlib
import matplotlib.pyplot as plt

#Import and seed random number generator
import random
from random import *

from datetime import datetime
seed(datetime.now())

#Get regular expression package
import re

#Import  Scikit learn framework
import sklearn as sk
from sklearn import svm
from sklearn import linear_model
from sklearn.metrics import roc_auc_score, roc_curve

F:\anaconda\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Import the functions built in previous parts
from Titanic_Import import *

full_set = pd.read_csv('D:/Datasets/Titanic/train.csv')
sub_set = pd.read_csv('D:/Datasets/Titanic/test.csv')

In [3]:
append_set = full_set
append_set = append_set.append([sub_set], ignore_index =True )
clean_set = Cleanse_Data_v3(append_set)
X_Train, Y_Train, X_CV, Y_CV, X_Test = dataset_splitter(clean_set, cv_size = 200)

Firstly lets define a loss function to fit.  For this I'll use a typical logistic loss function.

$J = - \frac{1}{m} \sum_{i=1}^{m}{(Y_i\log{\hat{Y_i}} + (1 - Y_i)\log{(1 - \hat{Y_i})}})$

In [4]:
def Log_Loss(Y_in, Y_hat_in) :
    epsilon = 1e-9
    m = Y_in.shape[0]
    
    Y_log = Y_in
    Y_hat_log = Y_hat_in
    
    #Y_log[np.where(Y_log == 0)] = -1
    #Y_hat_log[np.where(Y_hat_log == 0)] = -1
    
    loss = -(1.0/m )*np.sum((1 - Y_log)*np.log(1 - Y_hat_log + epsilon) + Y_log*np.log(Y_hat_log + epsilon))
    
    return loss

In [5]:
test = np.ones(200)
testlog = Log_Loss(Y_CV, test)
testlog

12.641192160147313

So to keep this somewhat striaghtforward I will only be using some basic mathematical functions - 

* Plus
* Minus
* Multiply
* Divide
* Min
* Max
* Log
* Sin
* Cos
* Tanh

And using a sigmoid activation output.

The general idea is to randomly select a feature and use a random one of the above functions on that output (or select a random number if it requires multiple inputs).

For random numbers I will be using a the absolute value gaussian distribution of $\mu = 1$ and $\sigma = 1$.

In [6]:
#Create list of function strings
test_func = []

test_func.append('np.add(')
test_func.append('np.subtract(')
test_func.append('np.multiply(')
test_func.append('np.divide(')
test_func.append('np.minimum(')
test_func.append('np.maximum(')
test_func.append('np.sin(')
test_func.append('np.cos(')
test_func.append('np.tanh(')

#Create function string mask to determine number of elements
#0 = no extra elements
#1 = 2 elements
#2 = 2 elements and +1e-9 to second element
func_mult = (1, 1, 1, 2, 1, 1, 0, 0, 0)

So now the functions are defined, I have also defined a mapping for functions that require 1 and 2 inputs, and a special function for divide that will add 1e-9 to avoid divide by zero errors.

In [7]:
def Generate_First_Gen(X_len, func_list, func_map):
    select_func = randint(0, len(func_map) - 1)
    mask= func_map[select_func]
    funct = func_list[select_func]
    
    column_selector_init = randint(0, X_len)
    if column_selector_init == 0 :
        rand_num1 = np.abs(gauss(1, 1))
        rand_num2 = np.abs(gauss(1, 1))
        
        if mask == 0:
            funcstr = funct + str(rand_num1) + ')'
        elif mask == 2:
            rand_num = np.abs(gauss(1, 1))
            funcstr = funct + str(rand_num2) + ','+ str(rand_num1) + '+1e-9)'
        elif mask == 1:  
            rand_num = np.abs(gauss(1, 1))
            funcstr = funct + str(rand_num2) + ','+ str(rand_num1) + ')'
        else :
            funcstr = 'ERROR'
        
    else :
        column_selector = column_selector_init - 1
  
        if mask == 0:
            funcstr = funct + 'X[:,' + str(column_selector) + '])'
        elif mask == 2:
            rand_num = np.abs(gauss(1, 1))
            funcstr = funct + 'X[:,' + str(column_selector) + '],'+ str(rand_num) + '+1e-9)'
        elif mask == 1:  
            rand_num = np.abs(gauss(1, 1))
            funcstr = funct + 'X[:,' + str(column_selector) + '],'+ str(rand_num) + ')'
        else :
            funcstr = 'ERROR'
    
    return funcstr

In [8]:
test_parent1 = Generate_First_Gen(5, test_func, func_mult)
test_parent2 = Generate_First_Gen(5, test_func, func_mult)

In [9]:
test_parent1

'np.tanh(X[:,0])'

Now with the first generation produced there needs to be a subsequent generation, this will take the output of 2 previous generations and put them together and apply some form of mutation.

So as there are two distinct scenarios for function selected - 
* Dual Input function
* Single Input function

##### Dual Input
The case of a dual input function being selected is no problem, simply apply the function to both parents.

##### Single Input
In the case of a single input function being selected then we can select a random parent and apply the function to that parent.

##### Mutation Schema
For a mutation schema, a simple idea is to have a random chance to call a first generation and recursively apply the algorithm to itself (after the first 2 parents have been processed).

In [10]:
def Generate_Next_Generation(X_len, parent1, parent2, mut_prob, func_list, func_mask):
    select_func = randint(0, len(func_mask) - 1)
    mask= func_mask[select_func]
    funct = func_list[select_func]
    
    mutate_rand = uniform(0.0, 1.0)
    
    if mutate_rand <= mut_prob :
        #Mutated outcome
        Mutation = Generate_First_Gen(X_len, func_list, func_mask)
        #Generate original non-mutated outcome
        Orig_output = Generate_Next_Generation(X_len, parent1, parent2, mut_prob, func_list, func_mask)
        #Generate final output
        funcstr = Generate_Next_Generation(X_len, Orig_output, Mutation, mut_prob, func_list, func_mask)
    
    else :
        #No Mutate outcome
        if mask == 0:
            #Single Input - choose random parent
            parent_rand = uniform(0.0, 1.0)
            if parent_rand <= 0.5 :
                funcstr = funct + parent1 + ')'
            else :
                funcstr = funct + parent2 + ')'
                
        elif mask == 2:
            #Divide
            rand_num = np.abs(gauss(1, 1))
            funcstr = funct +  parent1 + ' , ' + parent2 + '+1e-9)'
        elif mask == 1:  
            #Dual Input
            rand_num = np.abs(gauss(1, 1))
            funcstr =funct + parent1 + ' , ' + parent2  + ')'
        else :
            funcstr = 'ERROR'
    
    return funcstr

In [11]:
child_test = Generate_Next_Generation(5, test_parent1, test_parent2, 0.3, test_func, func_mult)

In [12]:
child_test

'np.sin(np.multiply(X[:,2],0.046544401782742995))'

So now we have the core functions built the next step will be to evaluate the performance of the candidate by executing the generated string, applying an activation function and then applying the loss function.

For simplicity I will use a simple sigmoid activation, and the cross-entropy loss as defined above.

In [13]:
#Sigmoid Activation Function - with overflow exception
def sigmoid_overf(z):
    if hasattr(z, "__len__") == True :
        z[np.where(z < -7e2)] = -4e-2
        
        
    else :
        if z <= -7e2:
            z = -4e-2

    s = 1/(1+np.exp(-z))  
    return s

#Redefined predictions due to tuple out of range errors
def normalize_predictions_new(y_hat):
    #y_out = y_hat.reshape((y_hat.shape[0],))
    y_out = np.around(y_hat)
    
    return y_out

def Generate_Predictions(childstr, X):
    return sigmoid_overf((eval(childstr)))

def Evaluate_Child(childstr, X, Y):
    y_hat = Generate_Predictions(childstr, X)
    #y_true = normalize_predictions_new(y_hat)
    return Log_Loss(Y, y_hat)


In [14]:
test_loss = Evaluate_Child(child_test, X_CV, Y_CV)

In [15]:
test_loss

0.6962387467819267

The next step is to put it all together and iterate over generations, picking the best parents from each generation and pairing them off and then picking the best function of any generation as the final output.


### Parameters Needed (due to dependancies)
* X
* Y
* Function list + mapping
* Probability of mutation

### New Parameters Needed
* Number of generations
* Number of children per generation
* Number of parents per generation
* Verbose (0,1)

I will also likely need 2 outputs - best children of generation and best children of all time.

For verbosity probably outputting the loss of the best child per generation should suffice.

Also note that while I have been generating this in parts with individual functions, due to the overlapping parameters I will be putting this into a class at the end.  This will have the benefits of parameter sharing as well as being able to include the option to initialize with a pre-defined population.

In [16]:
def Parent_selector(X, Y, gen_in, num_parents):
    #Initialize parameters
    num_in = len(gen_in)
    gen_out = []
    loss_out = []
    
    
    for i in range(num_in):
        #Get child and loss of child
        child_str = gen_in[i]
        child_loss = Evaluate_Child(child_str, X, Y)
        
        if len(gen_out) <= num_parents:
            #If not full generation out then append child
            gen_out.append(child_str)
            loss_out.append(child_loss)
        else :
            #Otherwise replace worst child if this child is better
            max_loss_out = max(loss_out)
            max_loss_out_ind = loss_out.index(max(loss_out))
            
            if child_loss < max_loss_out :
                del loss_out[max_loss_out_ind]
                del gen_out[max_loss_out_ind]
                
                gen_out.append(child_str)
                loss_out.append(child_loss)
    
    return gen_out, loss_out

def Best_Children_Selector(new_gen, new_losses, prev_gen, prev_losses) :
    #Set best as previous best
    best_childs = prev_gen
    best_losses = prev_losses
    
    #Initialize variables
    len_new = len(new_gen)
    
    #Iterate through new generation and replace worst of best if better
    for i in range(len_new):
        max_best_loss = max(best_losses)
        max_best_loss_ind = best_losses.index(max(best_losses))
        
        loss_this = new_losses[i]

        if loss_this < max_best_loss :
            del best_childs[max_best_loss_ind]
            del best_losses[max_best_loss_ind]
            
            best_childs.append(new_gen[i])
            best_losses.append(loss_this)
        
    return best_childs, best_losses




def Train_Genetic_Alg(X, Y, func_list, func_mask, mut_prob = 0.1, 
                      num_gens = 10, num_childs = 30, num_parents = 10, Verbose = 0, initial_gen = None) :
    
    #Define population containers
    generation_par = []
    hof_pars = []
    hof_losses = []
    hall_of_fame = {}
    
    #Initialize parameters
    X_len = X.shape[1]
    
    #Generate first generation
    if initial_gen == None :
        for i in range(num_parents):
            generation_par.append(Generate_First_Gen(X_len, func_list, func_mask))       
    else :
        generation_par = initial_gen
    
    
    #Iterate through generations
    for j in range(num_gens) :
        #Kill previous generation's children
        generation_out = []
        
        #Produce k children in generation j
        for k in range(num_childs):
            num_in = len(generation_par)
            
            #Pick random parents
            par1 = generation_par[randint(0, num_in - 1)]
            par2 = generation_par[randint(0, num_in - 1)]
            
            #Generate child
            childk = Generate_Next_Generation(X_len, par1, par2, mut_prob, func_list, func_mask)
            
            #Append to childs generation
            generation_out.append(childk)
        
        #Clear previous generations parents
        generation_par = []
        
        #Generate new parents
        generation_par, par_losses = Parent_selector(X, Y, generation_out, num_parents) 
        
        #Generate Hall of Fame
        if len(hof_pars) == 0:
            hof_pars, hof_losses = Parent_selector(X, Y, generation_out, 5)
        else :
            hof_pars, hof_losses = Best_Children_Selector(generation_par, par_losses, hof_pars, hof_losses) 
            
        #Show training progress
        if Verbose == 1:
            best_in_gen = min(par_losses)
            gen_avg = np.mean(par_losses)
            best_ever = min(hof_losses)
            print("Generation %s, BestGen=%s AverageGen=%s, BestEver=%s" % 
                  (j, best_in_gen, gen_avg, best_ever))
        
    
    return hof_pars, hof_losses

In [17]:
best_mods, mod_loss = Train_Genetic_Alg(X_Train, Y_Train, test_func, func_mult, mut_prob = 0.2, 
                      num_gens =12, num_childs = 100, num_parents = 10, Verbose = 1, initial_gen = None)

Generation 0, BestGen=0.6489265661253781 AverageGen=0.6769359453076063, BestEver=0.6489265661253781
Generation 1, BestGen=0.6493065087469346 AverageGen=0.654582499825159, BestEver=0.6489265661253781
Generation 2, BestGen=0.6102642621376481 AverageGen=0.6227725146328259, BestEver=0.6102642621376481
Generation 3, BestGen=0.5846908032913075 AverageGen=0.5939118062581066, BestEver=0.5846908032913075
Generation 4, BestGen=0.5820009578444209 AverageGen=0.586713263771836, BestEver=0.5820009578444209
Generation 5, BestGen=0.5820009578444209 AverageGen=0.5871509388209893, BestEver=0.5820009578444209
Generation 6, BestGen=0.5838954161318731 AverageGen=0.5859328675138422, BestEver=0.5820009578444209
Generation 7, BestGen=0.5820009578444209 AverageGen=0.5854977102031033, BestEver=0.5820009578444209
Generation 8, BestGen=0.5820009578444209 AverageGen=0.5847178176435102, BestEver=0.5820009578444209
Generation 9, BestGen=0.5820009578444209 AverageGen=0.5837875377343494, BestEver=0.5820009578444209
Ge

In [18]:
len(best_mods[0])

1203

In [19]:
train_pred = Generate_Predictions(best_mods[0], X_Train)
cv_pred = Generate_Predictions(best_mods[0], X_CV)

train_hat = normalize_predictions(train_pred)
cv_hat = normalize_predictions(cv_pred)

show_acc(Y_Train, train_hat)
show_acc(Y_CV, cv_hat)

Accuracy =  70.47756874095514
F1 Score =  0.5188679245283019

Confusion Matrix
       Labels  Actual True  Actual False
0   Pred True        110.0          50.0
1  Pred False        154.0         377.0
Accuracy =  73.0
F1 Score =  0.578125

Confusion Matrix
       Labels  Actual True  Actual False
0   Pred True         37.0          13.0
1  Pred False         41.0         109.0


So now we have a genetic algorithm constructor, there was a lot of hiccups and a lot of bugs in old code were brought to light, however this algorithm definitely generates solutions.

From testing a few areas of the code, the loss function has a major impact, especially the epsilon value to remove ln(0) errors and whether to normalize the predictions before applying the loss function or not.

However I would consider this a pretty interesting and successful experiment.

## Generating a Final Prediction

So as the final model is effectively the n best models, it seems logical to stack the models together into a voting schema as a simple method of ensembling.

And in generating this function it will be very useful in the future.

In [20]:
#Stack all models in output from best genetic algorithms
def Stack_models(hof_models, X):
    num_mods = len(hof_models)
    
    for i in range(num_mods):
        if i == 0 :
            pred = Generate_Predictions(hof_models[i], X)        
            stack_hat = normalize_predictions(pred)
        else :
            pred = Generate_Predictions(hof_models[i], X) 
            hats = normalize_predictions(pred)
            stack_hat = np.vstack((stack_hat, hats))
    
    return stack_hat

#Create simple voting mechanism from stack
def pred_from_stack(full_hat):
    num_mods = full_hat.shape[0]
    cutoff = num_mods / 2.0
    
    #Sum all votes
    out_hat = np.sum(full_hat, axis=0)
    
    #Allocate majority votes the correct label
    out_hat[np.where(out_hat <= cutoff)] = 0
    out_hat[np.where(out_hat > cutoff)] = 1
    
    return out_hat


In [21]:
full_y = Stack_models(best_mods, X_Train)
out_y = pred_from_stack(full_y)

out_y.shape

(691,)

In [22]:
show_acc(Y_Train, out_y)

Accuracy =  70.47756874095514
F1 Score =  0.5188679245283019

Confusion Matrix
       Labels  Actual True  Actual False
0   Pred True        110.0          50.0
1  Pred False        154.0         377.0


Now from all these parts its time to generate a full cohesive model class for this genetic binary classifier.

### Methods needed

* Initializer
* Loss function(s)
* Activation function(s)
* Generate Predictions
* Evaluate Child
* Generate initial generation
* Generate ith generation (i =/= 0)
* Parent Selector
* Best Child Selector (/Hall of fame generator)
* Train Models
* Stack Models
* Generate Predictions From Stack

In [31]:
#Genetic Algorithm Class
class Genetic_Binary_Classifier(object):
    def __init__(self, func_list, func_mask, eval_func = 1):
        self.func_list = func_list
        self.func_mask = func_mask
        self.best_models = None
        self.eval_func = eval_func
        
    def Log_Loss(self, Y_in, Y_hat_in) :
        epsilon = 1e-9
        m = Y_in.shape[0]
    
        Y_log = Y_in
        Y_hat_log = Y_hat_in
    
        #Y_log[np.where(Y_log == 0)] = -1
        #Y_hat_log[np.where(Y_hat_log == 0)] = -1
    
        loss = -(1.0/m )*np.sum((1 - Y_log)*np.log(1 - Y_hat_log + epsilon) + Y_log*np.log(Y_hat_log + epsilon))
    
        return loss
    
    #Calculate our accuracy figures
    def Get_acc_loss(self, Y_true, Y_hat) :
        #Compute inverse vectors for vectorization
        Y_true_inv = 1 - Y_true
        Y_hat_inv = 1 - Y_hat
        
        if hasattr(np.dot(Y_true,Y_hat.T), "__len__") == True :
            return 100
        if hasattr(np.dot(Y_true_inv,Y_hat_inv.T), "__len__") == True :  
            return 100
             
        acc = 100 - float((np.dot(Y_true,Y_hat.T) + np.dot(Y_true_inv,Y_hat_inv.T))/float(Y_true.size)*100)
    
        return acc

    #Sigmoid Activation Function - with overflow exception
    def sigmoid_overf(self, z):
        if hasattr(z, "__len__") == True :
            z[np.where(z < -7e2)] = -4e-2
        
        else :
            if z <= -7e2:
                z = -4e-2

        s = 1/(1+np.exp(-z))  
        return s

    #Redefined predictions due to tuple out of range errors
    def normalize_predictions_new(self, y_hat):
        #y_out = y_hat.reshape((y_hat.shape[0],))
        y_out = np.around(y_hat)
        
        return y_out
    
    #Generate first prediction level
    def Generate_Predictions(self, childstr, X):
        return self.sigmoid_overf((eval(childstr)))
    
    def Evaluate_Child(self, childstr, X, Y):
        y_hat = self.Generate_Predictions(childstr, X)
        #y_true = normalize_predictions_new(y_hat)
        return self.Log_Loss(Y, y_hat)
    
    def Evaluate_Child_v2(self, childstr, X, Y):
        y_hat = self.Generate_Predictions(childstr, X)
        y_true = self.normalize_predictions_new(y_hat)
        return self.Log_Loss(Y, y_true)
    
    def Evaluate_Child_v3(self, childstr, X, Y):
        y_hat = self.Generate_Predictions(childstr, X)
        y_true = self.normalize_predictions_new(y_hat)
        return self.Get_acc_loss(Y, y_true)

    #First Generation
    def Generate_First_Gen(self, X_len, func_list, func_map):
        select_func = randint(0, len(func_map) - 1)
        mask= func_map[select_func]
        funct = func_list[select_func]

        column_selector_init = randint(0, X_len)
        if column_selector_init == 0 :
            rand_num1 = np.abs(gauss(1, 1))
            rand_num2 = np.abs(gauss(1, 1))

            if mask == 0:
                funcstr = funct + str(rand_num1) + ')'
            elif mask == 2:
                rand_num = np.abs(gauss(1, 1))
                funcstr = funct + str(rand_num2) + ','+ str(rand_num1) + '+1e-9)'
            elif mask == 1:  
                rand_num = np.abs(gauss(1, 1))
                funcstr = funct + str(rand_num2) + ','+ str(rand_num1) + ')'
            else :
                funcstr = 'ERROR'

        else :
            column_selector = column_selector_init - 1
  
            if mask == 0:
                funcstr = funct + 'X[:,' + str(column_selector) + '])'
            elif mask == 2:
                rand_num = np.abs(gauss(1, 1))
                funcstr = funct + 'X[:,' + str(column_selector) + '],'+ str(rand_num) + '+1e-9)'
            elif mask == 1:  
                rand_num = np.abs(gauss(1, 1))
                funcstr = funct + 'X[:,' + str(column_selector) + '],'+ str(rand_num) + ')'
            else :
                funcstr = 'ERROR'

        return funcstr
    
    #Gen Subsequent Generation
    def Generate_Next_Generation(self, X_len, parent1, parent2, mut_prob, func_list, func_mask):
        select_func = randint(0, len(func_mask) - 1)
        mask= func_mask[select_func]
        funct = func_list[select_func]
        
        mutate_rand = uniform(0.0, 1.0)
        
        if mutate_rand <= mut_prob :
            #Mutated outcome
            Mutation = self.Generate_First_Gen(X_len, func_list, func_mask)
            #Generate original non-mutated outcome
            Orig_output = self.Generate_Next_Generation(X_len, parent1, parent2, mut_prob, func_list, func_mask)
            #Generate final output
            funcstr = self.Generate_Next_Generation(X_len, Orig_output, Mutation, mut_prob, func_list, func_mask)
        
        else :
            #No Mutate outcome
            if mask == 0:
                #Single Input - choose random parent
                parent_rand = uniform(0.0, 1.0)
                if parent_rand <= 0.5 :
                    funcstr = funct + parent1 + ')'
                else :
                    funcstr = funct + parent2 + ')'
                    
            elif mask == 2:
                #Divide
                rand_num = np.abs(gauss(1, 1))
                funcstr = funct +  parent1 + ' , ' + parent2 + '+1e-9)'
            elif mask == 1:  
                #Dual Input
                rand_num = np.abs(gauss(1, 1))
                funcstr =funct + parent1 + ' , ' + parent2  + ')'
            else :
                funcstr = 'ERROR'
        
        select_func = randint(0, len(func_mask) - 1)
        mask= func_mask[select_func]
        funct = func_list[select_func]
        
        mutate_rand = uniform(0.0, 1.0)
        
        if mutate_rand <= mut_prob :
            #Mutated outcome
            Mutation = self.Generate_First_Gen(X_len, func_list, func_mask)
            #Generate original non-mutated outcome
            Orig_output = self.Generate_Next_Generation(X_len, parent1, parent2, mut_prob, func_list, func_mask)
            #Generate final output
            funcstr = self.Generate_Next_Generation(X_len, Orig_output, Mutation, mut_prob, func_list, func_mask)
        return funcstr
    
    def Parent_selector(self, X, Y, gen_in, num_parents):
        #Initialize parameters
        num_in = len(gen_in)
        gen_out = []
        loss_out = []
        
        
        for i in range(num_in):
            #Get child and loss of child
            child_str = gen_in[i]
            if self.eval_func == 2 :
                child_loss = self.Evaluate_Child_v2(child_str, X, Y)
            elif self.eval_func == 3 :
                child_loss = self.Evaluate_Child_v3(child_str, X, Y)
            else:
                child_loss = self.Evaluate_Child(child_str, X, Y)
            
            if len(gen_out) <= num_parents:
                #If not full generation out then append child
                gen_out.append(child_str)
                loss_out.append(child_loss)
            else :
                #Otherwise replace worst child if this child is better
                max_loss_out = max(loss_out)
                max_loss_out_ind = loss_out.index(max(loss_out))
                
                if child_loss < max_loss_out :
                    del loss_out[max_loss_out_ind]
                    del gen_out[max_loss_out_ind]
                    
                    gen_out.append(child_str)
                    loss_out.append(child_loss)
        
        return gen_out, loss_out
    
    def Best_Children_Selector(self, new_gen, new_losses, prev_gen, prev_losses) :
        #Set best as previous best
        best_childs = prev_gen
        best_losses = prev_losses
        
        #Initialize variables
        len_new = len(new_gen)
        
        #Iterate through new generation and replace worst of best if better
        for i in range(len_new):
            max_best_loss = max(best_losses)
            max_best_loss_ind = best_losses.index(max(best_losses))
            
            loss_this = new_losses[i]
    
            if loss_this < max_best_loss :
                del best_childs[max_best_loss_ind]
                del best_losses[max_best_loss_ind]
                
                best_childs.append(new_gen[i])
                best_losses.append(loss_this)
            
        return best_childs, best_losses
    
    def Train_Genetic_Alg(self, X, Y, mut_prob = 0.1, num_gens = 10, num_childs = 30, 
                          num_parents = 10, Verbose = 0, initial_gen = None, Output = 0) :
        
        #Define population containers
        generation_par = []
        hof_pars = []
        hof_losses = []
        hall_of_fame = {}
        
        func_list = self.func_list
        func_mask = self.func_mask
        #Initialize parameters
        X_len = X.shape[1]
        
        #Generate first generation
        if self.best_models != None :
            generation_par = best_models
        elif initial_gen == None :
            for i in range(num_parents):
                generation_par.append(self.Generate_First_Gen(X_len, func_list, func_mask))       
        else :
            generation_par = initial_gen
        
        
        #Iterate through generations
        for j in range(num_gens) :
            #Kill previous generation's children
            generation_out = []
            
            #Produce k children in generation j
            for k in range(num_childs):
                num_in = len(generation_par)
                
                #Pick random parents
                par1 = generation_par[randint(0, num_in - 1)]
                par2 = generation_par[randint(0, num_in - 1)]
                
                #Generate child
                childk = self.Generate_Next_Generation(X_len, par1, par2, mut_prob, func_list, func_mask)
                
                #Append to childs generation
                generation_out.append(childk)
            
            #Clear previous generations parents
            generation_par = []
            
            #Generate new parents
            generation_par, par_losses = self.Parent_selector(X, Y, generation_out, num_parents) 
            
            #Generate Hall of Fame
            if len(hof_pars) == 0:
                hof_pars, hof_losses = self.Parent_selector(X, Y, generation_out, 5)
            else :
                hof_pars, hof_losses = self.Best_Children_Selector(generation_par, par_losses, hof_pars, hof_losses) 
                
            #Show training progress
            if Verbose == 1:
                best_in_gen = min(par_losses)
                gen_avg = np.mean(par_losses)
                best_ever = min(hof_losses)
                print("Generation %s, BestGen=%s AverageGen=%s, BestEver=%s" % 
                      (j, best_in_gen, gen_avg, best_ever))
            
            
        
        
        if Verbose == 1 :
            for p in range(len(hof_pars)):
                max_len = 0
                test_len = len(hof_pars[p])
                if p == 0 :
                    max_len = test_len
                if max_len < test_len:
                    max_len = test_len

            print("Largest Model = ", max_len)
            
        self.best_models = hof_pars
        
        if Output == 1:
            return hof_pars, hof_losses
        else:
            return None
    
        #Stack all models in output from best genetic algorithms
    def Stack_models(self, hof_models, X):
        num_mods = len(hof_models)
        
        for i in range(num_mods):
            if i == 0 :
                pred = self.Generate_Predictions(hof_models[i], X)        
                stack_hat = self.normalize_predictions_new(pred)
            else :
                pred = self.Generate_Predictions(hof_models[i], X) 
                hats = self.normalize_predictions_new(pred)
                stack_hat = np.vstack((stack_hat, hats))
        
        return stack_hat
    
    #Create simple voting mechanism from stack
    def pred_from_stack(self, full_hat):
        num_mods = full_hat.shape[0]
        cutoff = num_mods / 2.0
        
        #Sum all votes
        out_hat = np.sum(full_hat, axis=0)
        
        #Allocate majority votes the correct label
        out_hat[np.where(out_hat <= cutoff)] = 0
        out_hat[np.where(out_hat > cutoff)] = 1
        
        return out_hat
    
    #Generate Predictions from ensemble
    def Gen_Final_Preds(self, X):
        full_stack = self.Stack_models(self.best_models, X)
        out_preds = self.pred_from_stack(full_stack)
        return out_preds
        
    

In [32]:
#Create list of function strings
fin_func = []
fin_func.append('np.add(')
fin_func.append('np.subtract(')
fin_func.append('np.multiply(')
fin_func.append('np.divide(')
fin_func.append('np.minimum(')
fin_func.append('np.maximum(')
fin_func.append('np.sin(')
fin_func.append('np.cos(')
fin_func.append('np.tanh(')

#Create function string mask to determine number of elements
#0 = no extra elements
#1 = 2 elements
#2 = 2 elements and +1e-9 to second element
func_fin_msk = (1, 1, 1, 2, 1, 1, 0, 0,)

In [34]:
genetic_mod = Genetic_Binary_Classifier(fin_func, func_fin_msk,eval_func = 2)
genetic_mod.Train_Genetic_Alg(X_Train, Y_Train, mut_prob = 0.15, num_gens = 15, num_childs = 100, num_parents = 10, Verbose = 1)

Generation 0, BestGen=6.627846236606595 AverageGen=7.5466385780600795, BestEver=6.627846236606595
Generation 1, BestGen=5.57818733019831 AverageGen=6.453357483333528, BestEver=5.57818733019831
Generation 2, BestGen=5.158323767634997 AverageGen=5.646346999445601, BestEver=5.158323767634997
Generation 3, BestGen=4.858421222946916 AverageGen=5.420056897544595, BestEver=4.858421222946916
Generation 4, BestGen=4.738460205071683 AverageGen=4.975655854052256, BestEver=4.738460205071683
Generation 5, BestGen=4.228625879101944 AverageGen=4.730281044762008, BestEver=4.228625879101944
Generation 6, BestGen=4.228625879101944 AverageGen=4.561245065028725, BestEver=4.228625879101944
Generation 7, BestGen=4.228625879101944 AverageGen=4.4058410191449005, BestEver=4.228625879101944
Generation 8, BestGen=4.228625879101944 AverageGen=4.228625879101943, BestEver=4.228625879101944
Generation 9, BestGen=4.078674606757904 AverageGen=4.214993945252485, BestEver=4.078674606757904
Generation 10, BestGen=4.07867

In [35]:
gen_train = genetic_mod.Gen_Final_Preds(X_Train)
gen_CV = genetic_mod.Gen_Final_Preds(X_CV)

print('Training Resuts')  
show_acc(Y_Train, gen_train)
print('-------------------------------')
print('Cross Validation Resuts')  
show_acc(Y_CV, gen_CV)

Training Resuts
Accuracy =  80.4630969609262
F1 Score =  0.7169811320754716

Confusion Matrix
       Labels  Actual True  Actual False
0   Pred True        171.0          42.0
1  Pred False         93.0         385.0
-------------------------------
Cross Validation Resuts
Accuracy =  84.5
F1 Score =  0.7832167832167831

Confusion Matrix
       Labels  Actual True  Actual False
0   Pred True         56.0           9.0
1  Pred False         22.0         113.0


So this was informative as to how genetic algorithms work.  

From the above I'm still pretty sure there's a bug somewhere in the class implementation, however I have been unable to locate it.

From my own observations its interesting to note you can evaluate over any metric one chooses in this algorithm(in the use as a binary classifier), I chose logistic loss, normalized logistic loss and straight up accuracy and found that normalized logistic loss was generally generating the best algorithms overall.

This is also far from an optimized implementation, however this was not the point of the excerisize as adding in K-fold training and batch training are obvious immediate areas for improvement.

In terms of performance it was highly dependant upon the number of children/parents per generation and the number of generations ran, however due to the runtimes getting exponentially longer with more generations, however I was able to fairly consistently apprach the 80-85% on training/cross validation accuracy marks on some of the longer runs.  So overall this algorithm does perform okay, just at a heavy computational cost.

Also I did not experiment too much with changing the mutation parameter due to hitting the maximum recursion depth on values above ~0.3.

However using this programming paradigm to automatically generate and optimize different features of other machine learning algorithm could be very powerful, such as dynamically generating neural network architectures.  Unfortunately it is VERY computationally expensive and slow to run, however its definitely a useful tool to have.

In the future I will probably use a genetic programming framework (DEAP is the first obvious candidate) and use that implementation to dramatically speed up progress.

Next part to follow, although have yet to decide what to do.